# Implementasi Model Klasifikasi - Machine Learning
Berikut dataset yang akan digunakan untuk implementasi model klasifikasi:
https://www.kaggle.com/datasets/uciml/iris


In [1]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('https://raw.githubusercontent.com/DaffaZimraan/ML-Classification/main/IRIS_Train.csv')
test_data = pd.read_csv('https://raw.githubusercontent.com/DaffaZimraan/ML-Classification/main/IRIS_Test.csv')


## Decision Tree
Decision tree adalah model prediksi yang menggunakan struktur pohon atau hierarki keputusan. Dalam decision tree, setiap simpul mewakili sebuah atribut atau fitur, setiap cabang mewakili sebuah keputusan atau aturan, dan setiap daun dari pohon mewakili hasil akhir. Model ini sering digunakan dalam analisis keputusan dan pembelajaran mesin untuk membantu mengidentifikasi strategi yang paling mungkin untuk mencapai tujuan tertentu.

### A. Menggunakan GINI

### Tanpa Normalisasi Data

In [3]:
def calculate_metrics(conf_matrix):
    FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix)
    FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
    TP = np.diag(conf_matrix)
    TN = conf_matrix.sum() - (FP + FN + TP)

    precision = np.mean(TP / (TP + FP + np.finfo(float).eps))
    recall = np.mean(TP / (TP + FN + np.finfo(float).eps))
    f1_score = 2 * (precision * recall) / (precision + recall + np.finfo(float).eps)
    specificity = np.mean(TN / (TN + FP + np.finfo(float).eps))

    return precision, recall, f1_score, specificity

def create_confusion_matrix(actual, predicted, classes):
    class_dict = {cls: i for i, cls in enumerate(classes)}
    matrix = np.zeros((len(classes), len(classes)), dtype=int)
    for a, p in zip(actual, predicted):
        matrix[class_dict[a]][class_dict[p]] += 1
    return matrix

def gini_index(groups, classes):
    n_instances = float(sum([len(group) for group in groups]))
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] == class_val for row in group].count(True) / size
            score += p * p
        gini += (1.0 - score) * (size / n_instances)
    return gini

def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

train = train_data.values.tolist()
test = test_data.values.tolist()

tree = build_tree(train, max_depth=3, min_size=10)

predictions = [predict(tree, row) for row in test]

actual = [row[-1] for row in test]

classes = list(set(actual))
conf_matrix = create_confusion_matrix(actual, predictions, classes)

precision, recall, f1_score, specificity = calculate_metrics(conf_matrix)

accuracy = sum(1 for i in range(len(predictions)) if predictions[i] == actual[i]) / float(len(actual))

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)
print("Specificity:", specificity)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9
Precision: 0.923076923076923
Recall: 0.9
F1-Score: 0.911392405063291
Specificity: 0.9500000000000001
Confusion Matrix:
 [[10  0  0]
 [ 3  7  0]
 [ 0  0 10]]


### Dengan Normalisasi Data

In [4]:
def min_max_normalize(dataset):
    min_val = np.min(dataset, axis=0)
    max_val = np.max(dataset, axis=0)
    return (dataset - min_val) / (max_val - min_val + np.finfo(float).eps)

train_values = train_data.values
test_values = test_data.values
train_normalized = min_max_normalize(train_values[:, :-1])
train = np.column_stack((train_normalized, train_values[:, -1]))
test_normalized = min_max_normalize(test_values[:, :-1])
test = np.column_stack((test_normalized, test_values[:, -1]))

tree = build_tree(train.tolist(), max_depth=3, min_size=10)

predictions = [predict(tree, row[:-1]) for row in test.tolist()]

actual = [row[-1] for row in test.tolist()]

classes = list(set(actual))
conf_matrix = create_confusion_matrix(actual, predictions, classes)

precision, recall, f1_score, specificity = calculate_metrics(conf_matrix)

accuracy = sum(1 for i in range(len(predictions)) if predictions[i] == actual[i]) / float(len(actual))

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)
print("Specificity:", specificity)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 0.9
Precision: 0.923076923076923
Recall: 0.9
F1-Score: 0.911392405063291
Specificity: 0.9500000000000001
Confusion Matrix:
 [[10  0  0]
 [ 0 10  0]
 [ 3  0  7]]


### B. Menggunakan Entropy

### Tanpa Normalisasi Data

In [5]:
def entropy(groups, classes):
    total_instances = float(sum([len(group) for group in groups]))
    entropy = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] == class_val for row in group].count(True) / size
            if p > 0:
                score -= p * np.log2(p)
        entropy += (size / total_instances) * score
    return entropy

def test_split(index, value, dataset):
    left, right = [], []
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, float('inf'), None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            ent = entropy(groups, class_values)
            if ent < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], ent, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

def calculate_metrics(conf_matrix):
    FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix)
    FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
    TP = np.diag(conf_matrix)
    TN = conf_matrix.sum() - (FP + FN + TP)

    precision = np.mean(TP / (TP + FP + np.finfo(float).eps))
    recall = np.mean(TP / (TP + FN + np.finfo(float).eps))
    f1_score = 2 * (precision * recall) / (precision + recall + np.finfo(float).eps)
    specificity = np.mean(TN / (TN + FP + np.finfo(float).eps))

    return precision, recall, f1_score, specificity

def create_confusion_matrix(actual, predicted, classes):
    class_dict = {cls: i for i, cls in enumerate(classes)}
    matrix = np.zeros((len(classes), len(classes)), dtype=int)
    for a, p in zip(actual, predicted):
        matrix[class_dict[a]][class_dict[p]] += 1
    return matrix

train = train_data.values.tolist()
test = test_data.values.tolist()

tree = build_tree(train, max_depth=3, min_size=10)

predictions = [predict(tree, row) for row in test]

actual = [row[-1] for row in test]

classes = list(set(actual))
conf_matrix = create_confusion_matrix(actual, predictions, classes)

precision, recall, f1_score, specificity = calculate_metrics(conf_matrix)

accuracy = sum(1 for i in range(len(predictions)) if predictions[i] == actual[i]) / float(len(actual))

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)
print("Specificity:", specificity)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9
Precision: 0.923076923076923
Recall: 0.9
F1-Score: 0.911392405063291
Specificity: 0.9500000000000001
Confusion Matrix:
 [[10  0  0]
 [ 0 10  0]
 [ 3  0  7]]


### Dengan Normalisasi Data

In [6]:
train_values = train_data.values
test_values = test_data.values
train_normalized = min_max_normalize(train_values[:, :-1])
test_normalized = min_max_normalize(test_values[:, :-1])

train = np.column_stack((train_normalized, train_values[:, -1]))
test = np.column_stack((test_normalized, test_values[:, -1]))

tree = build_tree(train.tolist(), max_depth=3, min_size=10)

predictions = [predict(tree, row[:-1]) for row in test.tolist()]

actual = [row[-1] for row in test.tolist()]

classes = list(set(actual))
conf_matrix = create_confusion_matrix(actual, predictions, classes)

precision, recall, f1_score, specificity = calculate_metrics(conf_matrix)

accuracy = sum(1 for i in range(len(predictions)) if predictions[i] == actual[i]) / float(len(actual))

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)
print("Specificity:", specificity)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9
Precision: 0.923076923076923
Recall: 0.9
F1-Score: 0.911392405063291
Specificity: 0.9500000000000001
Confusion Matrix:
 [[10  0  0]
 [ 0 10  0]
 [ 3  0  7]]


## KNN
K-Nearest Neighbor (KNN) adalah salah satu algoritma pembelajaran mesin yang paling sederhana dan banyak digunakan untuk masalah klasifikasi dan regresi. Algoritma ini bekerja berdasarkan prinsip 'kesamaan fitur' (feature similarity), yang artinya, objek baru diklasifikasikan berdasarkan mayoritas dari K objek terdekatnya.

### A. Untuk K = 3


### Tanpa Normalisasi Data

In [7]:
def euclidean_distance(row1, row2):
    return np.sqrt(np.sum((np.array(row1) - np.array(row2)) ** 2))

def get_neighbors(train, test_row, num_neighbors):
    distances = []
    for train_row in train:
        dist = euclidean_distance(test_row[:-1], train_row[:-1])
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = [distances[i][0] for i in range(num_neighbors)]
    return neighbors

def predict_classification(train, test_row, num_neighbors):
    neighbors = get_neighbors(train, test_row, num_neighbors)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

def create_confusion_matrix(actual, predicted, classes):
    class_dict = {cls: i for i, cls in enumerate(classes)}
    matrix = np.zeros((len(classes), len(classes)), dtype=int)
    for a, p in zip(actual, predicted):
        matrix[class_dict[a]][class_dict[p]] += 1
    return matrix

def calculate_metrics(conf_matrix):
    FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix)
    FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
    TP = np.diag(conf_matrix)
    TN = conf_matrix.sum() - (FP + FN + TP)
    precision = np.mean(TP / (TP + FP + np.finfo(float).eps))
    recall = np.mean(TP / (TP + FN + np.finfo(float).eps))
    f1_score = 2 * (precision * recall) / (precision + recall + np.finfo(float).eps)
    specificity = np.mean(TN / (TN + FP + np.finfo(float).eps))
    return precision, recall, f1_score, specificity

train = train_data.values
test = test_data.values

num_neighbors = 3
predictions = [predict_classification(train, row, num_neighbors) for row in test]

actual = [row[-1] for row in test]

classes = list(set(actual))
conf_matrix = create_confusion_matrix(actual, predictions, classes)

precision, recall, f1_score, specificity = calculate_metrics(conf_matrix)

accuracy = sum(1 for i in range(len(predictions)) if predictions[i] == actual[i]) / float(len(actual))

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)
print("Specificity:", specificity)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9666666666666667
Precision: 0.9696969696969697
Recall: 0.9666666666666667
F1-Score: 0.9681794470526863
Specificity: 0.9833333333333334
Confusion Matrix:
 [[10  0  0]
 [ 0 10  0]
 [ 1  0  9]]


### Dengan Normalisasi Data

In [8]:
def min_max_normalize(dataset):
    min_val = np.min(dataset, axis=0)
    max_val = np.max(dataset, axis=0)
    return (dataset - min_val) / (max_val - min_val + np.finfo(float).eps)

train_values = train_data.values
test_values = test_data.values
train_normalized = min_max_normalize(train_values[:, :-1])
train = np.column_stack((train_normalized, train_values[:, -1]))
test_normalized = min_max_normalize(test_values[:, :-1])
test = np.column_stack((test_normalized, test_values[:, -1]))

num_neighbors = 3
predictions = [predict_classification(train, row, num_neighbors) for row in test]

actual = [row[-1] for row in test]

classes = list(set(actual))
conf_matrix = create_confusion_matrix(actual, predictions, classes)

precision, recall, f1_score, specificity = calculate_metrics(conf_matrix)

accuracy = sum(1 for i in range(len(predictions)) if predictions[i] == actual[i]) / float(len(actual))

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)
print("Specificity:", specificity)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9666666666666667
Precision: 0.9696969696969697
Recall: 0.9666666666666667
F1-Score: 0.9681794470526863
Specificity: 0.9833333333333334
Confusion Matrix:
 [[10  0  0]
 [ 0 10  0]
 [ 1  0  9]]


### B. Untuk K = 5

### Tanpa Normalisasi Data

In [9]:
train = train_data.values
test = test_data.values

num_neighbors = 5
predictions = [predict_classification(train, row, num_neighbors) for row in test]

actual = [row[-1] for row in test]

classes = list(set(actual))
conf_matrix = create_confusion_matrix(actual, predictions, classes)

precision, recall, f1_score, specificity = calculate_metrics(conf_matrix)

accuracy = sum(1 for i in range(len(predictions)) if predictions[i] == actual[i]) / float(len(actual))

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)
print("Specificity:", specificity)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9666666666666667
Precision: 0.9696969696969697
Recall: 0.9666666666666667
F1-Score: 0.9681794470526863
Specificity: 0.9833333333333334
Confusion Matrix:
 [[10  0  0]
 [ 0 10  0]
 [ 1  0  9]]


### Dengan Normalisasi Data

In [10]:
train_values = train_data.values
test_values = test_data.values
train_normalized = min_max_normalize(train_values[:, :-1])
train = np.column_stack((train_normalized, train_values[:, -1]))
test_normalized = min_max_normalize(test_values[:, :-1])
test = np.column_stack((test_normalized, test_values[:, -1]))

num_neighbors = 5
predictions = [predict_classification(train, row, num_neighbors) for row in test]

actual = [row[-1] for row in test]

classes = list(set(actual))
conf_matrix = create_confusion_matrix(actual, predictions, classes)

precision, recall, f1_score, specificity = calculate_metrics(conf_matrix)

accuracy = sum(1 for i in range(len(predictions)) if predictions[i] == actual[i]) / float(len(actual))

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)
print("Specificity:", specificity)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9666666666666667
Precision: 0.9696969696969697
Recall: 0.9666666666666667
F1-Score: 0.9681794470526863
Specificity: 0.9833333333333334
Confusion Matrix:
 [[10  0  0]
 [ 0 10  0]
 [ 1  0  9]]


### C. Untuk K = 7

### Tanpa Normalisasi Data

In [11]:
train = train_data.values
test = test_data.values

num_neighbors = 7
predictions = [predict_classification(train, row, num_neighbors) for row in test]

actual = [row[-1] for row in test]

classes = list(set(actual))
conf_matrix = create_confusion_matrix(actual, predictions, classes)

precision, recall, f1_score, specificity = calculate_metrics(conf_matrix)

accuracy = sum(1 for i in range(len(predictions)) if predictions[i] == actual[i]) / float(len(actual))

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)
print("Specificity:", specificity)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
Specificity: 1.0
Confusion Matrix:
 [[10  0  0]
 [ 0 10  0]
 [ 0  0 10]]


### Dengan Normalisasi Data

In [12]:
train_values = train_data.values
test_values = test_data.values
train_normalized = min_max_normalize(train_values[:, :-1])
train = np.column_stack((train_normalized, train_values[:, -1]))
test_normalized = min_max_normalize(test_values[:, :-1])
test = np.column_stack((test_normalized, test_values[:, -1]))

num_neighbors = 7
predictions = [predict_classification(train, row, num_neighbors) for row in test]

actual = [row[-1] for row in test]

classes = list(set(actual))
conf_matrix = create_confusion_matrix(actual, predictions, classes)

precision, recall, f1_score, specificity = calculate_metrics(conf_matrix)

accuracy = sum(1 for i in range(len(predictions)) if predictions[i] == actual[i]) / float(len(actual))

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)
print("Specificity:", specificity)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9666666666666667
Precision: 0.9696969696969697
Recall: 0.9666666666666667
F1-Score: 0.9681794470526863
Specificity: 0.9833333333333334
Confusion Matrix:
 [[10  0  0]
 [ 0 10  0]
 [ 1  0  9]]


## Naive-Bayes
Naive Bayes merupakan teknik klasifikasi statistik yang berdasarkan pada Teorema Bayes. Ini adalah keluarga dari algoritma klasifikasi yang sederhana namun sangat efisien dan berfungsi dengan baik dengan dataset besar. Meskipun sederhana, sering kali cukup efektif dan digunakan secara luas karena kecepatan dan skalabilitasnya.

### Tanpa Normalisasi Data

In [14]:
def calculate_prior(labels):
    classes = np.unique(labels)
    prior = {}
    for cls in classes:
        prior[cls] = len(labels[labels == cls]) / len(labels)
    return prior

def calculate_likelihood(features, label, cls):
    feature_values = features[label == cls]
    mean = np.mean(feature_values, axis=0)
    var = np.var(feature_values, axis=0)
    return mean, var

def fit_naive_bayes(features, labels):
    classes = np.unique(labels)
    model = {}
    for cls in classes:
        model[cls] = {}
        model[cls]['prior'] = calculate_prior(labels)[cls]
        mean, var = calculate_likelihood(features, labels, cls)
        model[cls]['mean'] = mean
        model[cls]['var'] = var
    return model

def predict_single(model, input_features):
    probabilities = {}
    for cls, parameters in model.items():
        total_features = len(parameters['mean'])
        probabilities[cls] = np.log(parameters['prior'])
        for index in range(total_features):
            mean = parameters['mean'][index]
            var = parameters['var'][index]
            x = input_features[index]
            probabilities[cls] += (-0.5 * np.log(2 * np.pi * var)) - ((x - mean) ** 2 / (2 * var))
    return max(probabilities, key=probabilities.get)

def predict(model, features):
    predictions = []
    for feature_set in features:
        predictions.append(predict_single(model, feature_set))
    return predictions

train_features = train_data.iloc[:, :-1].values
train_labels = train_data.iloc[:, -1].values
test_features = test_data.iloc[:, :-1].values
test_labels = test_data.iloc[:, -1].values

model = fit_naive_bayes(train_features, train_labels)

test_predictions = predict(model, test_features)

def confusion_matrix_manual(labels, predictions, classes):
    matrix = pd.DataFrame(np.zeros((len(classes), len(classes)), dtype=int), index=classes, columns=classes)
    for true, pred in zip(labels, predictions):
        matrix.loc[true, pred] += 1
    return matrix

conf_matrix = confusion_matrix_manual(test_labels, test_predictions, np.unique(test_labels))

def calculate_metrics(conf_matrix):
    metrics = {}
    for cls in conf_matrix.columns:
        tp = conf_matrix.loc[cls, cls]
        fn = conf_matrix.loc[cls].sum() - tp
        fp = conf_matrix[cls].sum() - tp
        tn = conf_matrix.sum().sum() - tp - fp - fn
        precision = tp / (tp + fp) if tp + fp != 0 else 0
        recall = tp / (tp + fn) if tp + fn != 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else 0
        specificity = tn / (tn + fp) if tn + fp != 0 else 0
        accuracy = (tp + tn) / conf_matrix.sum().sum()
        metrics[cls] = {
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score,
            'specificity': specificity,
            'accuracy': accuracy
        }
    return metrics

metrics = calculate_metrics(conf_matrix)

print("Accuracy:", np.mean([m['accuracy'] for m in metrics.values()]))
print("Precision:", np.mean([m['precision'] for m in metrics.values()]))
print("Recall:", np.mean([m['recall'] for m in metrics.values()]))
print("F1-Score:", np.mean([m['f1_score'] for m in metrics.values()]))
print("Specificity:", np.mean([m['specificity'] for m in metrics.values()]))
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9555555555555556
Precision: 0.9444444444444445
Recall: 0.9333333333333332
F1-Score: 0.9326599326599326
Specificity: 0.9666666666666667
Confusion Matrix:
                  Iris-setosa  Iris-versicolor  Iris-virginica
Iris-setosa               10                0               0
Iris-versicolor            0               10               0
Iris-virginica             0                2               8


### Dengan Normalisasi Data

In [16]:
def min_max_scaling(features):
    min_vals = np.min(features, axis=0)
    max_vals = np.max(features, axis=0)
    return (features - min_vals) / (max_vals - min_vals)

train_features = train_data.iloc[:, :-1].values
test_features = test_data.iloc[:, :-1].values

train_features_scaled = min_max_scaling(train_features)
test_features_scaled = min_max_scaling(test_features)

train_labels = train_data.iloc[:, -1].values
test_labels = test_data.iloc[:, -1].values

model = fit_naive_bayes(train_features_scaled, train_labels)
test_predictions = predict(model, test_features_scaled)

conf_matrix = confusion_matrix_manual(test_labels, test_predictions, np.unique(test_labels))

metrics = calculate_metrics(conf_matrix)

print("Accuracy:", np.mean([m['accuracy'] for m in metrics.values()]))
print("Precision:", np.mean([m['precision'] for m in metrics.values()]))
print("Recall:", np.mean([m['recall'] for m in metrics.values()]))
print("F1-Score:", np.mean([m['f1_score'] for m in metrics.values()]))
print("Specificity:", np.mean([m['specificity'] for m in metrics.values()]))
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9777777777777779
Precision: 0.9696969696969697
Recall: 0.9666666666666667
F1-Score: 0.9665831244778613
Specificity: 0.9833333333333334
Confusion Matrix:
                  Iris-setosa  Iris-versicolor  Iris-virginica
Iris-setosa               10                0               0
Iris-versicolor            0               10               0
Iris-virginica             0                1               9


## Support Vector Machine (SVM)

Support Vector Machine (SVM) adalah salah satu algoritma pembelajaran mesin yang paling populer dan kuat untuk klasifikasi biner, regresi, dan tugas-tugas lain yang serupa. SVM dirancang untuk menemukan hyperplane dalam ruang N-dimensi (N adalah jumlah fitur) yang secara jelas mengklasifikasikan data titik.

### A. Kernel Linear

### Tanpa Normalisasi Data

In [17]:
def load_data():
    train_data = pd.read_csv('https://raw.githubusercontent.com/DaffaZimraan/ML-Classification/main/IRIS_Train.csv')
    test_data = pd.read_csv('https://raw.githubusercontent.com/DaffaZimraan/ML-Classification/main/IRIS_Test.csv')
    return train_data, test_data

def preprocess_data(data):
    mapping = {species: idx for idx, species in enumerate(data['species'].unique())}
    data['species'] = data['species'].map(mapping)
    X = data.iloc[:, :-1].values
    y = data['species'].values
    y = np.where(y == 0, -1, 1)
    return X, y

def train_svm(X, y, learning_rate=0.001, lambda_param=0.01, epochs=1000):
    w = np.zeros(X.shape[1])
    b = 0
    n = len(y)
    for _ in range(epochs):
        for idx, x_i in enumerate(X):
            condition = y[idx] * (np.dot(x_i, w) - b) >= 1
            if condition:
                w -= learning_rate * (2 * lambda_param * w)
            else:
                w -= learning_rate * (2 * lambda_param * w - np.dot(x_i, y[idx]))
                b -= learning_rate * y[idx]
    return w, b

def predict(X, w, b):
    return np.sign(np.dot(X, w) - b)

def calculate_metrics(y_true, y_pred):
    tp = np.sum((y_pred == 1) & (y_true == 1))
    tn = np.sum((y_pred == -1) & (y_true == -1))
    fp = np.sum((y_pred == 1) & (y_true == -1))
    fn = np.sum((y_pred == -1) & (y_true == 1))

    precision = tp / (tp + fp + np.finfo(float).eps)
    recall = tp / (tp + fn + np.finfo(float).eps)
    f1_score = 2 * precision * recall / (precision + recall + np.finfo(float).eps)
    specificity = tn / (tn + fp + np.finfo(float).eps)

    return precision, recall, f1_score, specificity

def create_confusion_matrix(y_true, y_pred):
    cm = np.zeros((2, 2), dtype=int)
    for t, p in zip(y_true, y_pred):
        cm[int(t == 1), int(p == 1)] += 1
    return cm

train_data, test_data = load_data()
X_train, y_train = preprocess_data(train_data)
X_test, y_test = preprocess_data(test_data)

weights, bias = train_svm(X_train, y_train)

predictions = predict(X_test, weights, bias)

accuracy = np.mean(y_test == predictions)
precision, recall, f1_score, specificity = calculate_metrics(y_test, predictions)
conf_matrix = create_confusion_matrix(y_test, predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)
print("Specificity:", specificity)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
Specificity: 1.0
Confusion Matrix:
 [[10  0]
 [ 0 20]]


### Dengan Normalisasi Data

In [18]:
def min_max_normalize(X):
    min_val = np.min(X, axis=0)
    max_val = np.max(X, axis=0)
    return (X - min_val) / (max_val - min_val + np.finfo(float).eps)

def preprocess_data(data):
    mapping = {species: idx for idx, species in enumerate(data['species'].unique())}
    data['species'] = data['species'].map(mapping)
    X = data.iloc[:, :-1].values
    y = data['species'].values
    y = np.where(y == 0, -1, 1)
    X = min_max_normalize(X)
    return X, y

train_data, test_data = load_data()
X_train, y_train = preprocess_data(train_data)
X_test, y_test = preprocess_data(test_data)

weights, bias = train_svm(X_train, y_train)

predictions = predict(X_test, weights, bias)

accuracy = np.mean(y_test == predictions)
precision, recall, f1_score, specificity = calculate_metrics(y_test, predictions)
conf_matrix = create_confusion_matrix(y_test, predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)
print("Specificity:", specificity)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
Specificity: 1.0
Confusion Matrix:
 [[10  0]
 [ 0 20]]


### B. Kernel Polynomial

### Tanpa Normalisasi Data

In [19]:
def preprocess_data(data):
    mapping = {species: idx for idx, species in enumerate(data['species'].unique())}
    data['species'] = data['species'].map(mapping)
    X = data.iloc[:, :-1].values
    y = data['species'].values
    y = np.where(y == 0, -1, 1)
    return X, y

def polynomial_kernel(x, y, degree=3, coef=1):
    return (np.dot(x, y) + coef) ** degree

def train_svm(X, y, degree=3, coef=1, learning_rate=0.001, lambda_param=0.01, epochs=500):
    w = np.zeros(X.shape[1])
    b = 0
    for _ in range(epochs):
        for idx, x_i in enumerate(X):
            if y[idx] * (polynomial_kernel(x_i, w, degree, coef) + b) < 1:
                w += learning_rate * ((y[idx] * x_i) - (2 * lambda_param * w))
                b += learning_rate * y[idx]
            else:
                w -= learning_rate * (2 * lambda_param * w)
    return w, b

def predict(X, w, b, degree=3, coef=1):
    return np.sign(np.array([polynomial_kernel(x, w, degree, coef) for x in X]) + b)

def create_confusion_matrix(y_true, y_pred):
    classes = np.unique(y_true)
    cm = np.zeros((len(classes), len(classes)), int)
    for true, pred in zip(y_true, y_pred):
        cm[int(true == 1), int(pred == 1)] += 1
    return cm

def calculate_metrics(cm):
    tp = cm[1, 1]
    tn = cm[0, 0]
    fp = cm[0, 1]
    fn = cm[1, 0]
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_score = 2 * (precision * recall) / (precision + recall)
    specificity = tn / (tn + fp)
    return precision, recall, f1_score, specificity

train_data, test_data = load_data()
X_train, y_train = preprocess_data(train_data)
X_test, y_test = preprocess_data(test_data)

w, b = train_svm(X_train, y_train)
predictions = predict(X_test, w, b)

conf_matrix = create_confusion_matrix(y_test, predictions)
precision, recall, f1_score, specificity = calculate_metrics(conf_matrix)

accuracy = np.mean(predictions == y_test)
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1_score}")
print(f"Specificity: {specificity}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
Specificity: 1.0
Confusion Matrix:
[[10  0]
 [ 0 20]]


### Dengan Normalisasi Data

In [ ]:
def min_max_normalize(X):
    min_val = np.min(X, axis=0)
    max_val = np.max(X, axis=0)
    return (X - min_val) / (max_val - min_val + np.finfo(float).eps)

def preprocess_data(data):
    mapping = {species: idx for idx, species in enumerate(data['species'].unique())}
    data['species'] = data['species'].map(mapping)
    X = data.iloc[:, :-1].values
    y = data['species'].values
    y = np.where(y == 0, -1, 1)
    X = min_max_normalize(X)
    return X, y

train_data, test_data = load_data()
X_train, y_train = preprocess_data(train_data)
X_test, y_test = preprocess_data(test_data)

w, b = train_svm(X_train, y_train)
predictions = predict(X_test, w, b)

conf_matrix = create_confusion_matrix(y_test, predictions)
precision, recall, f1_score, specificity = calculate_metrics(conf_matrix)

accuracy = np.mean(predictions == y_test)
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1_score}")
print(f"Specificity: {specificity}")
print(f"Confusion Matrix:\n{conf_matrix}")


Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
Specificity: 1.0
Confusion Matrix:
[[10  0]
 [ 0 20]]


### C. Kernel RBF

### Tanpa Normalisasi Data

In [ ]:
def min_max_normalize(X):
    min_val = np.min(X, axis=0)
    max_val = np.max(X, axis=0)
    return (X - min_val) / (max_val - min_val + np.finfo(float).eps)

def preprocess_data(data, normalize=False):
    mapping = {species: idx for idx, species in enumerate(data['species'].unique())}
    data['species'] = data['species'].map(mapping)
    X = data.iloc[:, :-1].values
    y = data['species'].values
    y = np.where(y == 0, -1, 1)
    if normalize:
        X = min_max_normalize(X)
    return X, y

def rbf_kernel(x, y, gamma=1):
    return np.exp(-gamma * np.linalg.norm(x - y)**2)

def train_svm(X, y, kernel, gamma=1, learning_rate=0.001, lambda_param=0.01, epochs=500):
    w = np.zeros(X.shape[1])
    b = 0
    for _ in range(epochs):
        for idx, x_i in enumerate(X):
            if y[idx] * (np.dot(w, x_i) + b) < 1:
                w += learning_rate * (y[idx] * x_i - 2 * lambda_param * w)
            else:
                w -= learning_rate * (2 * lambda_param * w)
    return w, b

def predict(X, w, b):
    return np.sign(np.dot(X, w) + b)

def create_confusion_matrix(y_true, y_pred):
    classes = np.unique(y_true)
    cm = np.zeros((len(classes), len(classes)), int)
    for true, pred in zip(y_true, y_pred):
        cm[int(true == 1), int(pred == 1)] += 1
    return cm

def calculate_metrics(cm):
    tp = cm[1, 1]
    tn = cm[0, 0]
    fp = cm[0, 1]
    fn = cm[1, 0]
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_score = 2 * (precision * recall) / (precision + recall)
    specificity = tn / (tn + fp)
    return precision, recall, f1_score, specificity

train_data, test_data = load_data()
X_train, y_train = preprocess_data(train_data, normalize=False)
X_test, y_test = preprocess_data(test_data, normalize=False)

w, b = train_svm(X_train, y_train, rbf_kernel)
predictions = predict(X_test, w, b)

accuracy = np.mean(predictions == y_test)
conf_matrix = create_confusion_matrix(y_test, predictions)
precision, recall, f1_score, specificity = calculate_metrics(conf_matrix)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1_score}")
print(f"Specificity: {specificity}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
Specificity: 1.0
Confusion Matrix:
[[10  0]
 [ 0 20]]


### Dengan Normalisasi Data

In [ ]:
train_data, test_data = load_data()
X_train, y_train = preprocess_data(train_data, normalize=True)
X_test, y_test = preprocess_data(test_data, normalize=True)

w, b = train_svm(X_train, y_train, rbf_kernel)
predictions = predict(X_test, w, b)

accuracy = np.mean(predictions == y_test)
conf_matrix = create_confusion_matrix(y_test, predictions)
precision, recall, f1_score, specificity = calculate_metrics(conf_matrix)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1_score}")
print(f"Specificity: {specificity}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
Specificity: 1.0
Confusion Matrix:
[[10  0]
 [ 0 20]]


### D. Kernel Sigmoid

### Tanpa Normalisasi Data

In [ ]:
def sigmoid_kernel(x, y, gamma=0.1, coef0=1):
    return np.tanh(gamma * np.dot(x, y) + coef0)

def train_svm(X, y, kernel=sigmoid_kernel, gamma=0.1, coef0=1, learning_rate=0.001, lambda_param=0.01, epochs=500):
    w = np.zeros(X.shape[1])
    b = 0
    for _ in range(epochs):
        for idx, x_i in enumerate(X):
            condition = y[idx] * (np.dot(w, x_i) + b) < 1
            if condition:
                w += learning_rate * (y[idx] * x_i - 2 * lambda_param * w)
            else:
                w -= learning_rate * (2 * lambda_param * w)
    return w, b

def predict(X, w, b):
    return np.sign(np.dot(X, w) + b)

train_data, test_data = load_data()
X_train, y_train = preprocess_data(train_data, normalize=False)
X_test, y_test = preprocess_data(test_data, normalize=False)

w, b = train_svm(X_train, y_train, kernel=sigmoid_kernel)
predictions = predict(X_test, w, b)

conf_matrix = create_confusion_matrix(y_test, predictions)
precision, recall, f1_score, specificity = calculate_metrics(conf_matrix)
accuracy = np.mean(predictions == y_test)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall:}")
print(f"F1-Score: {f1_score}")
print(f"Specificity: {specificity}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
Specificity: 1.0
Confusion Matrix:
[[10  0]
 [ 0 20]]


### Dengan Normalisasi Data

In [ ]:
def sigmoid_kernel(x, y, gamma=0.1, coef0=1):
    return np.tanh(gamma * np.dot(x, y) + coef0)

def train_svm(X, y, kernel=sigmoid_kernel, gamma=0.1, coef0=1, learning_rate=0.001, lambda_param=0.01, epochs=500):
    w = np.zeros(X.shape[1])
    b = 0
    for _ in range(epochs):
        for idx, x_i in enumerate(X):
            condition = y[idx] * (np.dot(w, x_i) + b) < 1
            if condition:
                w += learning_rate * (y[idx] * x_i - 2 * lambda_param * w)
            else:
                w -= learning_rate * (2 * lambda_param * w)
    return w, b

def predict(X, w, b):
    return np.sign(np.dot(X, w) + b)

train_data, test_data = load_data()
X_train, y_train = preprocess_data(train_data, normalize=True)
X_test, y_test = preprocess_data(test_data, normalize=True)

w, b = train_svm(X_train, y_train, kernel=sigmoid_kernel)
predictions = predict(X_test, w, b)

conf_matrix = create_confusion_matrix(y_test, predictions)
precision, recall, f1_score, specificity = calculate_metrics(conf_matrix)
accuracy = np.mean(predictions == y_test)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall:}")
print(f"F1-Score: {f1_score}")
print(f"Specificity: {specificity}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
Specificity: 1.0
Confusion Matrix:
[[10  0]
 [ 0 20]]


## Artificial Neural Network (ANN)

ANN terdiri dari lapisan input, satu atau lebih lapisan tersembunyi, dan lapisan output. Setiap neuron dalam satu lapisan terhubung dengan neuron di lapisan berikutnya melalui bobot, yang merupakan parameter yang disesuaikan selama proses pembelajaran. Neuron ini mengumpulkan input, menerapkannya pada suatu fungsi (biasanya non-linear), dan mengirimkan output ke neuron berikutnya.

### Tanpa Normalisasi Data

In [23]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Precision, Recall

X_train = train_data.drop("species", axis=1).values
X_test = test_data.drop("species", axis=1).values

le = LabelEncoder()
y_train = le.fit_transform(train_data['species'])
y_test = le.transform(test_data['species'])

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = Sequential([
    Dense(6, input_dim=X_train.shape[1], activation='relu'),
    Dense(6, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1)

loss, accuracy, precision, recall = model.evaluate(X_test, y_test, verbose=0)

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

f1 = f1_score(y_true_classes, y_pred_classes, average='macro')

FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix)
TN = conf_matrix.sum() - (conf_matrix.sum(axis=0) - np.diag(conf_matrix)) - (conf_matrix.sum(axis=1) - np.diag(conf_matrix)) + np.diag(conf_matrix)
specificity = np.mean(TN / (TN + FP + np.finfo(float).eps))

print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')
print(f'Specificity: {specificity:.4f}')
print('Confusion Matrix:\n', conf_matrix)


Epoch 1/100
4/4 [==============================] - 1s 6ms/step - loss: 1.3300 - accuracy: 0.3500 - precision_3: 0.4348 - recall_3: 0.3333
Epoch 2/100
4/4 [==============================] - 0s 6ms/step - loss: 1.2518 - accuracy: 0.3917 - precision_3: 0.4762 - recall_3: 0.3333
Epoch 3/100
4/4 [==============================] - 0s 5ms/step - loss: 1.1913 - accuracy: 0.4833 - precision_3: 0.4938 - recall_3: 0.3333
Epoch 4/100
4/4 [==============================] - 0s 7ms/step - loss: 1.1344 - accuracy: 0.5750 - precision_3: 0.5000 - recall_3: 0.3333
Epoch 5/100
4/4 [==============================] - 0s 7ms/step - loss: 1.0874 - accuracy: 0.6250 - precision_3: 0.5181 - recall_3: 0.3583
Epoch 6/100
4/4 [==============================] - 0s 7ms/step - loss: 1.0402 - accuracy: 0.6500 - precision_3: 0.5402 - recall_3: 0.3917
Epoch 7/100
4/4 [==============================] - 0s 7ms/step - loss: 1.0069 - accuracy: 0.6667 - precision_3: 0.5556 - recall_3: 0.4167
Epoch 8/100
4/4 [=================

### Dengan Normalisasi Data

In [22]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Precision, Recall

X_train = train_data.drop("species", axis=1).values
X_test = test_data.drop("species", axis=1).values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

le = LabelEncoder()
y_train = le.fit_transform(train_data['species'])
y_test = le.transform(test_data['species'])

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = Sequential([
    Dense(6, input_dim=X_train.shape[1], activation='relu'),
    Dense(6, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1)

loss, accuracy, precision, recall = model.evaluate(X_test, y_test, verbose=0)

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

f1 = f1_score(y_true_classes, y_pred_classes, average='macro')

FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix)
TN = conf_matrix.sum() - (conf_matrix.sum(axis=0) - np.diag(conf_matrix)) - (conf_matrix.sum(axis=1) - np.diag(conf_matrix)) + np.diag(conf_matrix)
specificity = np.mean(TN / (TN + FP + np.finfo(float).eps))

print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')
print(f'Specificity: {specificity:.4f}')
print('Confusion Matrix:\n', conf_matrix)


Epoch 1/100
4/4 [==============================] - 2s 9ms/step - loss: 1.3118 - accuracy: 0.2000 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00
Epoch 2/100
4/4 [==============================] - 0s 8ms/step - loss: 1.2889 - accuracy: 0.2000 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00
Epoch 3/100
4/4 [==============================] - 0s 9ms/step - loss: 1.2663 - accuracy: 0.2000 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00
Epoch 4/100
4/4 [==============================] - 0s 7ms/step - loss: 1.2461 - accuracy: 0.2000 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00
Epoch 5/100
4/4 [==============================] - 0s 8ms/step - loss: 1.2256 - accuracy: 0.1917 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00
Epoch 6/100
4/4 [==============================] - 0s 9ms/step - loss: 1.2078 - accuracy: 0.2167 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00
Epoch 7/100
4/4 [==============================] - 0s 8ms/step - loss: 1.1892 - accuracy: 0.2083 - precision_2: 0.0000e+00 -